# Data Science - Laboratorio 5
## Minería de Texto: Clasificación de tweets 
- - -
### Integrantes:
- Diego Alberto Leiva 21752
- José Pablo Orellana 21970
- - -

## Librerias

NOTA: Importante tener el Token de la API en el folder `C:\Users\<Windows-username>\.kaggle\kaggle.json`

Para una guia mas detallada o si tiene otro sistema dirigase a: https://www.kaggle.com/docs/api#getting-started-installation-&-authentication 

Asimismo puede visitar la documentacion oficial de la API: https://github.com/Kaggle/kaggle-api/tree/main/docs#api-credentials

In [1]:
# Kaggle API
import kaggle
from kaggle.rest import ApiException

# Operaciones con archivos
import os
import zipfile

## Carga de Dataset

### Descargar el archivo con Kaggle

Puede omitir esta celda si decide descargar los archivos manualmente desde https://www.kaggle.com/c/nlp-getting-started/data

In [2]:
# Autenticación con Kaggle API
kaggle.api.authenticate()

# Definir competencia y directorio de descarga
competition_name = 'nlp-getting-started'
target_path = 'data'

# Descargar archivos de la competencia
try:
    kaggle.api.competition_download_files(competition='nlp-getting-started',
                                          path='data',
                                          quiet=False,
                                          force=False)
    
    print(f"EXITO: Archivos descargados en '{target_path}'")

except ApiException as e:
    print(f"ERROR al llamar KaggleApi -> competition_download \n\t{e}")
    print("Puede que necesite aceptar las reglas de la competencia en https://www.kaggle.com/c/nlp-getting-started/rules")

100%|██████████| 593k/593k [00:00<00:00, 1.06MB/s]


EXITO: Archivos descargados en 'data'


### Descomprimir el Zip

Puede omitir esta celda si ya tiene los archivos descomprimidos

In [3]:
try:
    # Verificar si se descargó un archivo .zip y descomprimirlo
    if os.path.exists(f"{target_path}/{competition_name}.zip"):
        # Descomprimir archivo .zip
        with zipfile.ZipFile(f'{target_path}/{competition_name}.zip', 'r') as zip_ref:
            zip_ref.extractall(target_path)

        # Eliminar archivo .zip
        os.remove(f'{target_path}/{competition_name}.zip')
        os.remove(f'{target_path}/sample_submission.csv')
        print(f"EXITO: Archivo .zip descomprimido en '{target_path}'")
    else:
        Exception(f"No se encontró archivo .zip en '{target_path}'")
except Exception as e:
    print(f"ERROR al descomprimir archivo .zip \n\t{e}")

EXITO: Archivo .zip descomprimido en 'data'
